
# ResearchTrack2 Assignment 1.2

## Libraries and initialization

In [ ]:
import ipywidgets as widgets
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox

%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import rospy
import tf
import actionlib
import actionlib.msg
import sys
import select
import assignment_2_2022.msg 
from std_srvs.srv import *
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Point, Pose, Twist
from assignment_2_2022.msg import Position_velocity
from sensor_msgs.msg import LaserScan
from tf.transformations import quaternion_matrix
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation

# initialize the node 
rospy.init_node('user_input')

# initialize the publisher to send a msg (using the velocity and the position as parameters)
pub = rospy.Publisher("/Position_velocity", Position_velocity, queue_size = 1)

# initialize the global variable necessary for the cell below
actual_pos_x = 0
actual_pos_y = 0

goal_cancelled = 0
goal_set = 0

min_distance = 0


## Action Client

In [ ]:
# create the action client
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)
# wait for the server to be started
client.wait_for_server()

## Insert the goal and print the actual pose and the goals set and cancelled

In [ ]:
print("ENTER THE COORDINATES (x,y): ")
positionX = widgets.FloatText(description = "x", layout=Layout(width='25%', align="center", grid_area='pX'))
positionY = widgets.FloatText(description = "y", layout=Layout(width='25%', align="center", grid_area='pY'))
button = widgets.Button(description="Send", button_style='success', layout=Layout(width='25%', align="center",
grid_area='b1'), icon='check')
button2 = widgets.Button(description="Cancel", button_style='danger', layout=Layout(width='25%', align="center",
grid_area='b2'))
box = VBox([HBox([positionX, positionY]), HBox([button, button2])])
display(box)

text_box_set = widgets.IntText(description="N° set goals", value=goal_set, disabled=True)
text_box_canc = widgets.IntText(description="N° can goals", value=goal_cancelled, disabled=True)

def send_goal(b):
    global goal_set
    global text_box_set, goal
    # create the goal to send to the server
    goal = assignment_2_2022.msg.PlanningGoal()
    goal.target_pose.pose.position.x = positionX.value
    goal.target_pose.pose.position.y = positionY.value
    vis.draw_point(positionX.value, positionY.value, 'blue')
    # send the goal to the action server
    client.send_goal(goal)
    goal_set += 1
    text_box_set.value = goal_set
    
def cancel_goal(b):
    global goal_cancelled
    global text_box_canc
    client.cancel_goal()
    goal_cancelled += 1
    text_box_canc.value = goal_cancelled 
    
def disegna_grafico(valore1, valore2):
    labels = ['Set', 'Cancelled']
    mycolors = ["green", "red"]
    sizes = [valore1, valore2]
    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors = mycolors)
    ax.axis('equal')
    plt.title('Goals')
    plt.show()
    
interactive_plot = widgets.interactive(disegna_grafico, valore1=text_box_set, valore2=text_box_canc)
display(interactive_plot)
    
button.on_click(send_goal)
button2.on_click(cancel_goal)

## Print the trajectory of the robot and the goals

In [ ]:
class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ko')
        self.x_data, self.y_data = [] , []
    def plot_init(self):
        self.ax.set_xlim(10, -10)
        self.ax.set_ylim(10, -10)
        self.ax.grid(True)
        return self.ln
    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln
    def draw_point(self, x, y, color):
        self.ax.plot(x, y, color, marker='o')
    def add_legend(self):
        legend_labels = ['Goal_canc', 'Goal_reached', 'Next_goal']
        colors = ['red', 'green', 'blue']
        legend_elements = [
            plt.Line2D([0], [0], marker='o', color=color, label=label, linestyle='None')
            for color, label in zip(colors, legend_labels)
        ]
        self.ax.legend(handles=legend_elements)
        
vis = Visualiser()


def publish_values(msg):
    
    global positionX,positionY,vis
    # get the postion from the message
    pos = msg.pose.pose.position

    # get the twist from the message
    velocity = msg.twist.twist.linear

    # create custom message (of the type corresponding to the file in the msg folder)
    position_and_velocity = Position_velocity()

    # assign the parameters of the custom message
    position_and_velocity.x = pos.x
    position_and_velocity.y = pos.y
    position_and_velocity.v_x = velocity.x
    position_and_velocity.v_y = velocity.y
    
    global actual_pos_x
    global actual_pos_y
    actual_pos_x = pos.x
    actual_pos_y = pos.y
    text_box.value = f'{actual_pos_x:.6f}'
    text_box2.value = f'{actual_pos_y:.6f}'
    
    

    # Publish the custom message
    pub.publish(position_and_velocity)

            
    
rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)

def result(msg):
    global positionX,positionY,vis
    # get the status of the result from the message
    status = msg.status.status

    # if status is equal 2, the goal was preempted (cancelled)
    if status == 2:
        vis.draw_point(positionX.value, positionY.value, 'red')
    # if status is equal 3, the goal was reached
    elif status == 3:
        vis.draw_point(positionX.value, positionY.value, 'green')

    
sub_result = rospy.Subscriber('/reaching_goal/result', assignment_2_2022.msg.PlanningActionResult, result)
vis.add_legend()
plt.show(block=True)
#subscribe to the topic /odom
text_box = widgets.FloatText(description="Position x", value=actual_pos_x, disabled=True, continuous_update=True)
text_box2 = widgets.FloatText(description="Position y", value=actual_pos_y, disabled=True, continuous_update=True)
rospy.Subscriber("/odom", Odometry, publish_values)
print("ACTUAL POSE OF THE ROBOT: ")
display(text_box, text_box2)

### Compute the distance from the closest obastacle

In [ ]:
text_box3 = widgets.FloatText(description="", value=min_distance, disabled=True)

def min_obstacle_distance(msg):
    global min_distance
    counter=0
    min_distance = msg.ranges[0]
    for i in msg.ranges:
        if i<min_distance:
            min_distance = i
            
            text_box3.value = f'{min_distance:.6f}'
                
            

### Print the distance from the closest obastacle

In [ ]:
rospy.Subscriber("/scan", LaserScan, min_obstacle_distance)
print("DISTANCE BETWEEN THE ACTUAL POSE AND THE NEAREST OBSTACLE: ")
display(text_box3)